In [1]:
import dolfinx
from dolfinx import fem, mesh, io, plot, log
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace, Expression )
from dolfinx.fem import FunctionSpace, assemble_matrix, assemble_vector, apply_lifting, set_bc
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import VTXWriter, XDMFFile, VTKFile


import gmsh
from dolfinx.io import gmshio



# For MPI-based parallelization
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# PETSc solvers
from petsc4py import PETSc


import ufl
from ufl import (TestFunction, TestFunctions, TrialFunction, Identity, grad, det, div, dev, inv, tr, sqrt, conditional ,\
                 gt, dx, inner, derivative, dot, ln, split, sin, cos, pi)



import basix
from basix.ufl import element, mixed_element

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
plt.close('all')


log.set_log_level(log.LogLevel.WARNING)

In [2]:
Length = 10.0
Width  = 10.0
n_x, n_y = 20, 20
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([0, 0]), np.array([Length, Width])], [n_x, n_y], mesh.CellType.quadrilateral)


x = ufl.SpatialCoordinate(domain)
mu     = Constant(domain, PETSc.ScalarType(8.0e7))
lmbda  = Constant(domain, PETSc.ScalarType(4.0e7))
# V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

In [3]:
def left(x):      ### Marked as Boundary 1
    return np.isclose(x[0], 0)
    
def right(x):     ### Marked as Boundary 2
    return np.isclose(x[0], Length)
    
def top(x):       ### Marked as Boundary 3
    return np.isclose(x[1], Width)

def bottom(x):    ### Marked as Boundary 4
    return np.isclose(x[1], 0)


fdim = domain.topology.dim - 1

left_facets   = mesh.locate_entities_boundary(domain, fdim, left)
right_facets  = mesh.locate_entities_boundary(domain, fdim, right)
top_facets    = mesh.locate_entities_boundary(domain, fdim, top)
bottom_facets = mesh.locate_entities_boundary(domain, fdim, bottom)


# Concatenate and sort the arrays based on facet indices.
marked_facets = np.hstack([left_facets, right_facets, top_facets, bottom_facets])

marked_values = np.hstack([np.full_like(left_facets, 1), 
                           np.full_like(right_facets, 2), 
                           np.full_like(top_facets, 3), 
                           np.full_like(bottom_facets, 4)])

sorted_facets = np.argsort(marked_facets)
facet_tags = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])


metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tags, metadata=metadata)
dv = ufl.Measure('dx', domain=domain, metadata=metadata)

#  Define facet normal
Normal = ufl.FacetNormal(domain)

In [4]:
# Define function space, both vectorial and scalar 
U2     = element("Lagrange", domain.basix_cell(), 2, shape=(2,))  # For displacement
P1     = element("Lagrange", domain.basix_cell(), 1)              # For Lagrange Multiplier to enforce incompressibility


TH = mixed_element([U2, P1])      # Taylor-Hood style mixed element
ME = functionspace(domain, TH)            # Total space for all DOFs

# Define actual functions with the required DOFs
w    = Function(ME)
u, p = split(w)  

# A copy of functions to store values in the previous step
w_old         = Function(ME)
u_old, p_old = split(w_old)   

# Define test functions 
w_test = TestFunction(ME)

# Define trial functions needed for automatic differentiation
dw = TrialFunction(ME)   

In [5]:
def F(u):
    I = ufl.Identity(len(u))
    F = I + ufl.grad(u)
    return ufl.variable(F)

def C_tensor(F):
    return ufl.variable(F.T * F)

def J(F):
    return ufl.det(F)

def Ic(u):
    C_tensor = ufl.variable(C(u))
    return ufl.tr(C_tensor)
    
def Incompressible_neo_Hookean_u(F, p):
    C = C_tensor(F)
    I1 = ufl.tr(C)
    Jval = J(F)
    return (mu / 2) * (I1 - 2) - p * (Jval - 1)



In [6]:
stretch  = 1.02      # stretch amplitude

dispTot  = (stretch-1)*Length
rate     = 1.e-1
Ttot     = (stretch-1)/rate
numSteps = 100
dt       = Ttot/numSteps  # (fixed) step size
t        = 0.0

def dispRamp(t):
    return dispTot*t/Ttot

In [7]:
########################################################################
############ Dirichlet Boundary Conditions for Displacement ############
########################################################################

# Find the specific DOFs which will be constrained.
LeftBC_u1_dofs  = fem.locate_dofs_topological(ME.sub(0).sub(0), facet_tags.dim, facet_tags.find(1))
LeftBC_u2_dofs  = fem.locate_dofs_topological(ME.sub(0).sub(1), facet_tags.dim, facet_tags.find(1))
RightBC_u1_dofs = fem.locate_dofs_topological(ME.sub(0).sub(0), facet_tags.dim, facet_tags.find(2))
RightBC_u2_dofs = fem.locate_dofs_topological(ME.sub(0).sub(1), facet_tags.dim, facet_tags.find(2))


# Dirichlet BCs: u1, u2 on Left BC are fixed. u2 on the Right BC is fixed
LeftBC_Fixed_u1        = dirichletbc(0.0, LeftBC_u1_dofs,  ME.sub(0).sub(0))     # u1 fix - Left BC
LeftBC_Fixed_u2        = dirichletbc(0.0, LeftBC_u2_dofs,  ME.sub(0).sub(1))     # u2 fix - Left BC
RightBC_Fixed_u2       = dirichletbc(0.0, RightBC_u2_dofs, ME.sub(0).sub(1))     # u2 fix - Right BC



# Constant for applied displacement. Dirichlet BCs: u1 on the Right BC is variable
disp_cons_u              = Constant(domain,PETSc.ScalarType(dispRamp(0)))
RightBC_Deformation_u1   = dirichletbc(disp_cons_u, RightBC_u1_dofs, ME.sub(0).sub(0))  # disp ramp for u1 - Right BC



#########################################################################
################### Boundary Conditions and Weak Form ###################
#########################################################################

BoundaryConditions = [LeftBC_Fixed_u1, LeftBC_Fixed_u2, RightBC_Fixed_u2, RightBC_Deformation_u1]

In [8]:
#### Weak Form
F_ = F(u)
Total_Energy = Incompressible_neo_Hookean_u(F_, p) * dv

First_variation  = ufl.derivative(Total_Energy, w, w_test)
Jacobian         = ufl.derivative(First_variation, w, dw)
problem = NonlinearProblem(First_variation, w, BoundaryConditions, Jacobian)

# the global newton solver and params
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6      # 1e-8
solver.atol = 1e-6      # 1e-8
solver.max_it = 100      # 50
solver.report = True


#  The Krylov solver parameters.
ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly" # "preonly" works equally well
opts[f"{option_prefix}pc_type"] = "lu" # do not use 'gamg' pre-conditioner
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
opts[f"{option_prefix}ksp_max_it"] = 30      # 30
ksp.setFromOptions()

In [9]:
# # results file name
results_name = "2D_uniaxial_tension_Incompressible-neo-Hookean"

# # # Function space for projection of results
U1     = element("DG", domain.basix_cell(), 1, shape=(2,))   # For displacement
P0     = element("DG", domain.basix_cell(), 1)               # For  pressure  
V2       = fem.functionspace(domain, U1)             #Vector function space
VP       = fem.functionspace(domain, P0)             #Scalar function space

# fields to write to output file
u_vis = Function(V2)
p_vis = Function(VP)
u_vis.name = "Displacement"
p_vis.name = "Lagrange Multiplier"

vtk_u = VTKFile(domain.comm, "results/neo-Hookean/Incompressible/Displacement/u.vtk", "w")
vtk_p = VTKFile(domain.comm, "results/neo-Hookean/Incompressible/Lagrange-Multiplier/p.vtk", "w")


def writeResults(t):
    
    u_vis.interpolate(w.sub(0))
    p_vis.interpolate(w.sub(1))

    vtk_u.write_function(u_vis, t)
    vtk_p.write_function(p_vis, t)

pointForDisp = np.array([Length, Width, 0.0], dtype=np.float64)

bb_tree = dolfinx.geometry.bb_tree(domain,domain.topology.dim)
cell_candidates = dolfinx.geometry.compute_collisions_points(bb_tree, pointForDisp)
colliding_cells = dolfinx.geometry.compute_colliding_cells(domain, cell_candidates, pointForDisp).array
 

In [10]:
# Give the step a descriptive name
step = "Stretch"

# Variables for storing time history
totSteps = numSteps+1

# Iinitialize a counter for reporting data
ii=0

# Write initial state to file
# writeResults(t=0.0)   

print("------------------------------------")
print("Simulation Start")
print("------------------------------------")

startTime = datetime.now()
dt = 1
Ttot = 101
# Time-stepping solution procedure loop
while (round(t + dt, 9) <= Ttot):
# for t in range(0, 101, 1):
     
    # # increment time
    t += dt 
    # # increment counter
    ii += 1
    
    # update time variables in time-dependent BCs 
    disp_cons_u.value    = dispRamp(t)

    
    # Solve the problem
    try:
        (iter, converged) = solver.solve(w)
    except: # Break the loop if solver fails
        print("Ended Early")
        break
    
    # Collect results from MPI ghost processes
    w.x.scatter_forward()
    
    ## Write output to file
    writeResults(t)
    
    # Update DOFs for next step
    w_old.x.array[:] = w.x.array
    print(f"Step: {step} | Increment: {ii}, Iterations: {iter}")
   
    ## Print progress of calculation
    # if ii%1 == 0:      
        # now = datetime.now()
        # current_time = now.strftime("%H:%M:%S")
        # print(f"Step: {step} | Increment: {ii}, Iterations: {iter}")
        # print(f"Simulation Time: {round(t,4)} s  of  {Ttot} s")  
        # print()
    

## close the output file.
# file_results.close()
         
## End analysis
print("-----------------------------------------")
print("End computation")                 

print("------------------------------------------")
print(f"Elapsed real time:  {datetime.now() - startTime}")
print("------------------------------------------")

------------------------------------
Simulation Start
------------------------------------
Step: Stretch | Increment: 1, Iterations: 3
Step: Stretch | Increment: 2, Iterations: 3
Step: Stretch | Increment: 3, Iterations: 3
Step: Stretch | Increment: 4, Iterations: 3
Step: Stretch | Increment: 5, Iterations: 3
Step: Stretch | Increment: 6, Iterations: 3
Step: Stretch | Increment: 7, Iterations: 3
Step: Stretch | Increment: 8, Iterations: 3
Step: Stretch | Increment: 9, Iterations: 3
Step: Stretch | Increment: 10, Iterations: 3
Step: Stretch | Increment: 11, Iterations: 3
Step: Stretch | Increment: 12, Iterations: 3
Step: Stretch | Increment: 13, Iterations: 3
Step: Stretch | Increment: 14, Iterations: 3
Step: Stretch | Increment: 15, Iterations: 3
Step: Stretch | Increment: 16, Iterations: 3
Step: Stretch | Increment: 17, Iterations: 3
Step: Stretch | Increment: 18, Iterations: 3
Step: Stretch | Increment: 19, Iterations: 3
Step: Stretch | Increment: 20, Iterations: 3
Step: Stretch | In